Heavily adapted from: 
https://github.com/tobyatgithub/bert_tutorial/blob/master/Bert_tutorial1_embeddings.ipynb

We will be using pretrained BERT model to go from raw words into latent embeddings
#### word -> tokens -> ids -> hidden states -> embeddings

In [ ]:
!conda install pytorch torchvision -c pytorch
!pip install pytorch_pretrained_bert

In [14]:
import pandas as pd
import nlp
import nltk
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:
labels = nlp.load_labels('articles_db.db')
corpus = nlp.load_articles('articles_db.db')
df = pd.DataFrame(corpus, columns=['title', 'body'])

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 636850.96B/s]


In [16]:
single_article = corpus[0][1]
single_article = corpus[0][1]

In [20]:
# word -> tokens
raw_text_a = corpus[0][1]
raw_text_b = corpus[1][1]
text_a = "[CLS] " + single_article + " [SEP]" # notice the space
text_b = raw_text_b + " [SEP]" # don't need [CLS] for the second sentence
tokens_a = tokenizer.tokenize(text_a)
tokens_b = tokenizer.tokenize(text_b)

In [21]:
# word -> tokens -> ids -> hidden states -> embeddings

tokens = []
input_type_ids = []
# masks for segment, 0 for the first sentence, 1 for the second sentence.
# use 1 if there's only one sentence.

for token in tokens_a:
    tokens.append(token)
    input_type_ids.append(0)

if tokens_b:
    for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
        
print("tokens:", tokens)   
print("type_ids:", input_type_ids)

tokens: ['[CLS]', 'portsmouth', ',', 'england', '—', 'eddie', 'i', '##zza', '##rd', ',', 'one', 'of', 'britain', '’', 's', 'most', 'famous', 'comedians', ',', 'is', 'on', 'a', 'mani', '##c', ',', 'multi', '-', 'city', 'tour', 'to', 'save', 'the', 'united', 'kingdom', 'from', 'becoming', 'an', 'isolation', '##ist', 'state', '.', 'but', 'he', 'began', 'a', 'recent', 'debate', 'with', 'charm', 'and', 'bon', '##hom', '##ie', ',', 'encouraging', 'everyone', 'to', 'vote', 'in', 'the', 'european', 'union', 'referendum', 'thursday', ',', 'regardless', 'of', 'their', 'views', '.', '“', 'i', '’', 'm', 'trying', 'to', 'encourage', 'people', 'to', 'vote', '‘', 'remain', '’', 'on', 'the', '23rd', ',', 'but', 'if', 'you', 'vote', '‘', 'leave', ',', '’', 'fine', '”', 'said', 'i', '##zza', '##rd', ',', 'speaking', 'this', 'month', 'to', 'a', 'packed', 'lecture', 'at', 'the', 'university', 'of', 'portsmouth', '.', 'wearing', 'a', 'pink', ',', 'french', '-', 'style', 'be', '##ret', 'decorated', 'with', 

In [22]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)
for pair in zip(tokens, input_ids):
    print(pair)
# notice the case ---> uncased

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2230 > 512). Running this sequence through BERT will result in indexing errors


('[CLS]', 101)
('portsmouth', 10913)
(',', 1010)
('england', 2563)
('—', 1517)
('eddie', 5752)
('i', 1045)
('##zza', 20715)
('##rd', 4103)
(',', 1010)
('one', 2028)
('of', 1997)
('britain', 3725)
('’', 1521)
('s', 1055)
('most', 2087)
('famous', 3297)
('comedians', 25119)
(',', 1010)
('is', 2003)
('on', 2006)
('a', 1037)
('mani', 23624)
('##c', 2278)
(',', 1010)
('multi', 4800)
('-', 1011)
('city', 2103)
('tour', 2778)
('to', 2000)
('save', 3828)
('the', 1996)
('united', 2142)
('kingdom', 2983)
('from', 2013)
('becoming', 3352)
('an', 2019)
('isolation', 12477)
('##ist', 2923)
('state', 2110)
('.', 1012)
('but', 2021)
('he', 2002)
('began', 2211)
('a', 1037)
('recent', 3522)
('debate', 5981)
('with', 2007)
('charm', 11084)
('and', 1998)
('bon', 14753)
('##hom', 23393)
('##ie', 2666)
(',', 1010)
('encouraging', 11434)
('everyone', 3071)
('to', 2000)
('vote', 3789)
('in', 1999)
('the', 1996)
('european', 2647)
('union', 2586)
('referendum', 9782)
('thursday', 9432)
(',', 1010)
('regardle

In [23]:
# padding
seq_length = 30 # max allowed length & padding length for each pair of sentences. 512
input_mask = [1] * len(input_ids)
print(input_ids)
print(input_mask)
print(input_type_ids)
while len(input_ids) < seq_length:
    input_ids.append(0)
    input_mask.append(0)
    input_type_ids.append(0)
    
print()
print(input_ids)
print(input_mask)
print(input_type_ids)

[101, 10913, 1010, 2563, 1517, 5752, 1045, 20715, 4103, 1010, 2028, 1997, 3725, 1521, 1055, 2087, 3297, 25119, 1010, 2003, 2006, 1037, 23624, 2278, 1010, 4800, 1011, 2103, 2778, 2000, 3828, 1996, 2142, 2983, 2013, 3352, 2019, 12477, 2923, 2110, 1012, 2021, 2002, 2211, 1037, 3522, 5981, 2007, 11084, 1998, 14753, 23393, 2666, 1010, 11434, 3071, 2000, 3789, 1999, 1996, 2647, 2586, 9782, 9432, 1010, 7539, 1997, 2037, 5328, 1012, 1523, 1045, 1521, 1049, 2667, 2000, 8627, 2111, 2000, 3789, 1520, 3961, 1521, 2006, 1996, 13928, 1010, 2021, 2065, 2017, 3789, 1520, 2681, 1010, 1521, 2986, 1524, 2056, 1045, 20715, 4103, 1010, 4092, 2023, 3204, 2000, 1037, 8966, 8835, 2012, 1996, 2118, 1997, 10913, 1012, 4147, 1037, 5061, 1010, 2413, 1011, 2806, 2022, 13465, 7429, 2007, 2586, 2990, 1998, 1041, 1012, 1057, 1012, 23433, 1010, 2002, 2794, 2007, 1037, 24639, 2868, 1024, 1523, 2021, 1045, 2052, 2360, 2000, 1996, 1520, 2681, 1521, 7206, 2008, 1996, 6830, 2003, 2006, 1996, 13386, 1012, 1524, 1996, 4245, 

In [24]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
#model = model.cuda()
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

100%|██████████| 407873900/407873900 [00:17<00:00, 23358384.53B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [27]:
with torch.no_grad():
    # ids -> hidden state vectors
    input_tensor = torch.LongTensor(input_ids).view(-1,1)
    input_mask = torch.LongTensor(input_mask).view(-1,1)
    input_type_ids = torch.LongTensor(input_type_ids).view(-1,1)
    
    print(input_tensor.shape)
    print(input_mask.shape)  
    print(input_type_ids.shape)
    encoded_layers, _ = model(input_tensor, token_type_ids=input_type_ids, attention_mask=input_mask)

torch.Size([2230, 1])
torch.Size([2230, 1])
torch.Size([2230, 1])


In [28]:
# to get the token embedding vector, we can sum the last four
print(text_a, text_b)
sum_last_four = torch.sum(torch.stack(encoded_layers[-4:]), dim=0)
print(sum_last_four.shape)

[CLS] PORTSMOUTH, England — Eddie Izzard, one of Britain’s most famous comedians, is on a manic, multi-city tour to save the United Kingdom from becoming an isolationist state. But he began a recent debate with charm and bonhomie, encouraging everyone to vote in the European Union referendum Thursday, regardless of their views.
“I’m trying to encourage people to vote ‘remain’ on the 23rd, but if you vote ‘leave,’ fine” said Izzard, speaking this month to a packed lecture at the University of Portsmouth.
Wearing a pink, French-style beret decorated with Union Jack and E.U. badges, he added with a wry smile: “But I would say to the ‘leave’ voters that the voting is on the 24th.”
The generational divide is a major fault line in the E.U. referendum, with the young more pro-E.U. than their elderly counterparts. Campaigning on all sides has been temporarily suspended in the wake of the shooting of British lawmaker Jo Cox on Thursday. But as it resumes, campaigners will seek to energize their

In [29]:
print(torch.cat(encoded_layers[-4:]).shape)


torch.Size([8920, 1, 768])


In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dakaspar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
def process_document(text):
	"""
	Processes a text document by coverting all words to lower case,
	tokenizing, removing all non-alphabetical characters,
	and stemming each word.
	Args:
		text: A string of the text of a single document.
	Returns:
		A list of processed words from the document.
	"""
	# Convert words to lower case
	text = text.lower()

	# Tokenize corpus and remove all non-alphabetical characters
	tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
	tokens = tokenizer.tokenize(text)

	# Remove stopwords
	stop_words = nltk.corpus.stopwords.words('english')
	set_stopwords = set(stop_words)
	stopwords_removed = [token for token in tokens if not token in set_stopwords]

	# Stem words
	stemmer = nltk.stem.SnowballStemmer('english')
	stemmed = [stemmer.stem(word) for word in stopwords_removed]

	# Return list of processed words
	return stemmed

In [ ]:
df = pd.DataFrame(corpus, columns=['title', 'body'])

In [12]:
process_document(corpus[0][1])

['portsmouth',
 'england',
 'eddi',
 'izzard',
 'one',
 'britain',
 'famous',
 'comedian',
 'manic',
 'multi',
 'citi',
 'tour',
 'save',
 'unit',
 'kingdom',
 'becom',
 'isolationist',
 'state',
 'began',
 'recent',
 'debat',
 'charm',
 'bonhomi',
 'encourag',
 'everyon',
 'vote',
 'european',
 'union',
 'referendum',
 'thursday',
 'regardless',
 'view',
 'tri',
 'encourag',
 'peopl',
 'vote',
 'remain',
 '23rd',
 'vote',
 'leav',
 'fine',
 'said',
 'izzard',
 'speak',
 'month',
 'pack',
 'lectur',
 'univers',
 'portsmouth',
 'wear',
 'pink',
 'french',
 'style',
 'beret',
 'decor',
 'union',
 'jack',
 'e',
 'u',
 'badg',
 'ad',
 'wri',
 'smile',
 'would',
 'say',
 'leav',
 'voter',
 'vote',
 '24th',
 'generat',
 'divid',
 'major',
 'fault',
 'line',
 'e',
 'u',
 'referendum',
 'young',
 'pro',
 'e',
 'u',
 'elder',
 'counterpart',
 'campaign',
 'side',
 'temporarili',
 'suspend',
 'wake',
 'shoot',
 'british',
 'lawmak',
 'jo',
 'cox',
 'thursday',
 'resum',
 'campaign',
 'seek',
 'e

In [13]:
corpus[0][1]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dakaspar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'PORTSMOUTH, England —\xa0Eddie Izzard, one of Britain’s most famous comedians, is on a manic, multi-city tour to save the United Kingdom from becoming an isolationist state. But he began a recent debate with charm and bonhomie, encouraging everyone to vote in the European Union referendum Thursday, regardless of their views.\n“I’m trying to encourage people to vote ‘remain’ on the 23rd, but if you vote ‘leave,’ fine” said Izzard, speaking this month to a packed lecture at the University of Portsmouth.\nWearing a pink, French-style beret decorated with Union Jack and E.U. badges, he added with a wry smile: “But I would say to the ‘leave’ voters that the voting is on the 24th.”\nThe generational divide is a major fault line in the E.U. referendum, with the young more pro-E.U. than their elderly counterparts. Campaigning on all sides has been temporarily suspended in the wake of the shooting of British lawmaker Jo Cox on Thursday. But as it resumes, campaigners will seek to energize thei